<div style="background-color: #f0f8ff; border-left: 6px solid #2196F3; padding: 15px; border-radius: 5px;">

# 🚀 Analyse Exploratoire du Marché Immobilier Français (2021-2025)

**Objectif :** Ce notebook effectue une analyse exploratoire des données de Demandes de Valeurs Foncières (DVF) sur 5 ans. L'objectif est de fournir à notre persona, **Alexandre Dubois**, une première visualisation des grandes tendances du marché pour guider ses futurs investissements.

**Méthodologie :**
* **1. Configuration :** Définition des constantes et des schémas de données pour une performance optimale.
* **2. Chargement Optimisé :** Lecture des 5 fichiers CSV volumineux (2021-2025) en utilisant des techniques d'optimisation (Parquet).
* **3. Nettoyage Robuste :** Nettoyage et ingénierie des features (calcul du `prix_m2`).
* **4. Dashboard Statique (Persona) :** Visualisation des tendances macro (évolution sur 5 ans, types de biens).
* **5. Carte Interactive :** Exploration géographique des prix et des volumes.
* **6. Outils d'Export :** Fonctions pour exporter les analyses.

</div>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox, Layout
from IPython.display import display, HTML, clear_output
import warnings
import os
from tqdm.auto import tqdm  # Pour les barres de progression
from scipy import stats

# Configuration de l'environnement
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
# Utilisation d'un template Plotly plus professionnel
px.defaults.template = "plotly_white"

print("✅ Bibliothèques importées et environnement configuré.")

<div style="background-color: #fffbea; border-left: 6px solid #ffeb3b; padding: 15px; border-radius: 5px;">

## 💡 2. Définition des Constantes (Optimisation)

C'est la cellule la **plus importante pour la performance.**

Pour charger des Go de données sans saturer la RAM, nous devons être explicites :

* `FILE_PATHS` : Liste des 5+ fichiers CSV sources (nos "ingrédients bruts").
* `COLUMNS_TO_LOAD` : Nous ne chargeons *que* les colonnes vitales pour notre analyse.
* `DTYPE_MAP` : C'est l'optimisation clé. Nous indiquons à Pandas le type de données de chaque colonne *avant* le chargement pour diviser l'usage mémoire.
</div>

In [ ]:

FILE_PATHS = [
    
    "dvf_geolocalisees_2021.csv",
    "dvf_geolocalisees_2022.csv",
    "dvf_geolocalisees_2023.csv",
    "dvf_geolocalisees_2024.csv",
    "dvf_geolocalisees_2025.csv"
]

# --- 2. COLONNES ESSENTIELLES
# On a supprimé 'nom_commune' et 'code_commune' -> Gain de RAM massif
COLUMNS_TO_LOAD = [
    'date_mutation', 
    'valeur_fonciere', 
    'code_departement', 
    'surface_reelle_bati', 
    'nombre_pieces_principales', 
    'type_local', 
    'latitude', 
    'longitude'
]

# --- 3. SCHÉMA DE DONNÉES (CORRIGÉ) ---
DTYPE_MAP = {
    'valeur_fonciere': 'float32',
    'code_departement': 'category', # Sera chargé en category (ou 'object' si échec)
    'surface_reelle_bati': 'float32',
    'nombre_pieces_principales': 'float32', # Corrigé
    'type_local': 'category', # Sera chargé en category (ou 'object' si échec)
    'latitude': 'float32',
    'longitude': 'float32'
    # On ne met pas 'nom_commune' ou 'code_commune'
}

print("✅ Constantes de configuration ")
print(f"Nous allons charger {len(COLUMNS_TO_LOAD)} colonnes depuis {len(FILE_PATHS)} fichiers.")

<div style="background-color: #f0fff4; border-left: 6px solid #4CAF50; padding: 15px; border-radius: 5px;">

## ⚙️ 3. Fonctions de Chargement et Nettoyage

Nous modularisons le chargement et le nettoyage en deux fonctions distinctes pour plus de clarté.

### 3.1 Fonction `load_dvf_data` 
`load_dvf_data()` itère sur chaque fichier CSV source, et charge chaque fichier *par morceaux* (`chunksize`). Cela garantit que nous ne chargeons jamais le fichier entier en mémoire d'un coup.
Par conséquent elle ne plante pas si une colonne de `COLUMNS_TO_LOAD` est manquante dans un chunk.

### 3.2 Fonction `clean_dvf_data`
`clean_dvf_data()` prend le DataFrame brut et applique la logique de nettoyage :
1.  Convertit `date_mutation` en `datetime`.
2.  Supprime les lignes critiques manquantes.
3.  Calcule `prix_m2` et filtre les **outliers** (le cœur de l'analyse).
4.  Crée les colonnes temporelles (`annee`, `trimestre`) et les convertit en `category` pour économiser encore plus de mémoire.

</div>

In [ ]:
# NOUVELLE CELLULE 7 (Fonctions avec nettoyage robuste)

def load_dvf_data(file_paths, use_cols, dtypes):
    """
    Charge plusieurs fichiers CSV DVF volumineux de manière optimisée 
    en utilisant des chunks et des dtypes spécifiés.
    """
    print(f"Chargement de {len(file_paths)} fichiers...")
    all_chunks = []
    
    # Barre de progression pour les fichiers
    for file_path in tqdm(file_paths, desc="Progression Fichiers"):
        try:
            # On spécifie les dtypes à lire
            # on_bad_lines='skip' ignore les lignes corrompues
            chunk_iter = pd.read_csv(
                file_path,
                dtype=dtypes, 
                chunksize=100_000,
                low_memory=False,
                on_bad_lines='skip'
            )
            
            for chunk in chunk_iter:
                # On ne garde que les colonnes qui nous intéressent ET qui existent
                cols_qui_existent = [col for col in use_cols if col in chunk.columns]
                chunk_filtre = chunk[cols_qui_existent]
                all_chunks.append(chunk_filtre)
                
        except FileNotFoundError:
            print(f"\n/!\\ ALERTE: Fichier non trouvé: {file_path}. Il sera ignoré.")
        except Exception as e:
            print(f"\n/!\\ ERREUR lors de la lecture de {file_path}: {e}")

    if not all_chunks:
        print("❌ ERREUR: Aucun chunk n'a été chargé. Vérifiez vos chemins de fichiers.")
        return pd.DataFrame()

    print("Concatenation de tous les chunks...")
    df_raw = pd.concat(all_chunks, ignore_index=True, sort=False)
    print("✅ Chargement terminé.")
    return df_raw

def clean_dvf_data(df):
    """
    Nettoie le DataFrame DVF brut pour l'analyse.
    VERSION ROBUSTE : Force les conversions en category à la fin.
    """
    if df.empty:
        return pd.DataFrame()
        
    print("🔧 Nettoyage et préparation des données (v2)...")
    initial_rows = len(df)
    
    # 1. Conversion de la date (essentiel)
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
    
    # 2. Suppression des lignes avec données critiques manquantes
    colonnes_critiques = [
        'date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 
        'latitude', 'longitude', 'type_local', 'code_departement'
    ]
    colonnes_a_verifier = [col for col in colonnes_critiques if col in df.columns]
    df = df.dropna(subset=colonnes_a_verifier)
    
    # 3. Calcul du prix au m² et filtrage des aberrations
    df = df[
        (df['valeur_fonciere'] >= 1000) & 
        (df['surface_reelle_bati'] >= 9)
    ]
    
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    
    p_01 = df['prix_m2'].quantile(0.01)
    p_99 = df['prix_m2'].quantile(0.99)
    
    df = df[
        (df['prix_m2'] >= p_01) & 
        (df['prix_m2'] <= p_99)
    ]
    
    # 4. Ingénierie des features temporelles
    df['annee'] = df['date_mutation'].dt.year
    df['trimestre'] = df['date_mutation'].dt.quarter
    
    # --- OPTIMISATION FINALE (LA CLÉ) ---
    # On force la conversion des colonnes qui auraient dû être 'category'
    print("   -> Forçage des types 'category' pour optimiser la RAM...")
    cols_to_categorize = ['code_departement', 'type_local', 'annee', 'trimestre']
    for col in cols_to_categorize:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    final_rows = len(df)
    rows_dropped = initial_rows - final_rows
    print(f"✅ Nettoyage terminé. {rows_dropped:,} lignes aberrantes ou inutiles supprimées.")
    print(f"   {final_rows:,} transactions valides restantes pour l'analyse.")
    
    return df

print("✅ Fonctions  de chargement et de nettoyage définies.")

<div style="background-color: #f4f4f4; border-left: 6px solid #607D8B; padding: 15px; border-radius: 5px;">

## ⚡ 4. Exécution du Chargement (Logique Parquet)

C'est ici que nous appelons les fonctions définies ci-dessus:

1.  Elle cherche d'abord le fichier `dvf_clean_2021-2025.parquet`.
2.  **Si le fichier existe :** Le chargement est **quasi-instantané** .
3.  **Si le fichier n'existe pas :**
    * Elle exécute le chargement lent depuis les CSV (Mode Initialisation).
    * Elle **crée le fichier Parquet** pour que la prochaine exécution soit instantanée.

L'information clé à la fin est **`memory usage`** : elle prouve l'efficacité de nos optimisations de `dtype`.

</div>

In [ ]:

PARQUET_FILE_PATH = 'dvf_clean_2021-2025.parquet'
if os.path.exists(PARQUET_FILE_PATH):
    # CAS 1: Le fichier Parquet existe (RAPIDE: 3 secondes)
    print(f"✅ Fichier Parquet trouvé ! Chargement ultra-rapide depuis '{PARQUET_FILE_PATH}'...")
    df_clean = pd.read_parquet(PARQUET_FILE_PATH)
    print("   -> Chargement terminé.")

else:
    # CAS 2: Première exécution (LENT: 1 minute, UNE SEULE FOIS)
    print(f" Fichier Parquet non trouvé. Lancement du premier chargement (lent) depuis les CSV...")
    
    # --- Exécution du chargement (votre ancien code) ---
    df_raw = load_dvf_data(FILE_PATHS, COLUMNS_TO_LOAD, DTYPE_MAP)

    if not df_raw.empty:
        df_clean = clean_dvf_data(df_raw)
        
        # --- ÉTAPE CRUCIALE: SAUVEGARDE AU FORMAT PARQUET ---
        try:
            print(f"Sauvegarde des {len(df_clean):,} lignes nettoyées au format Parquet...")
            df_clean.to_parquet(PARQUET_FILE_PATH, index=False)
            print(f"   -> ✅ Sauvegarde terminée ! ('{PARQUET_FILE_PATH}')")
            print("   -> Lors de la prochaine exécution, le chargement sera instantané.")
        except Exception as e:
            print(f" /!\\ Erreur lors de la sauvegarde Parquet : {e}")
            print("     Vérifiez que 'pyarrow' est bien installé (pip install pyarrow)")
            
    else:
        print("❌ Aucune donnée n'a été chargée. Le notebook ne peut pas continuer.")
        df_clean = pd.DataFrame(columns=COLUMNS_TO_LOAD + ['prix_m2', 'annee', 'trimestre'])

# --- Affichage final (commun aux deux cas) ---
if not df_clean.empty:
    # On affiche un ÉCHANTILLON, pas .head(), pour éviter de surcharger le notebook
    print("\n--- Aperçu aléatoire des données ---")
    display(df_clean.sample(5))
    
    # Afficher le résumé mémoire (la preuve de l'optimisation)
    print("\n--- Informations et Utilisation Mémoire (Optimisée) ---")
    df_clean.info(memory_usage='deep')
else:
    print("❌ Aucune donnée n'est disponible pour l'analyse.")

<div style="background-color: #f9f0ff; border-left: 6px solid #9C27B0; padding: 15px; border-radius: 5px;">

## 📊 5. Dashboard Statique (Analyse Persona)

Avant de plonger dans l'interactivité, nous répondons aux questions de base d'Alexandre (*Besoins I et II*) :

1.  **Comment les prix ont-ils évolué sur 5 ans ?** (Tendance)
2.  **Le marché est-il dynamique ?** (Volume des transactions)
3.  **Quels types de biens sont vendus ?** (Composition du marché)
4.  **Comment se situe sa région (Rhône) par rapport aux autres ?** (Contexte local)
</div>

In [ ]:
# NOUVELLE CELLULE 11 (Dashboard avec Sampling)

def create_persona_dashboard(df):
    """
    Crée un dashboard 2x2 répondant aux questions clés du persona.
    VERSION RAPIDE : Utilise un échantillon pour le box plot.
    """
    if df.empty:
        print("❌ Pas de données pour le dashboard persona.")
        return

    print("📊 Création du dashboard d'analyse stratégique (Rapide)...")
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "1. Tendance des Prix (Médian) sur 5 ans",
            "2. Dynamisme du Marché (Volume des Ventes)",
            "3. Composition du Marché (Types de biens)",
            f"4. Focus Local: Prix/m² (Rhône vs. France)"
        ),
        specs=[
            [{"type": "xy"}, {"type": "xy"}],
            [{"type": "domain"}, {"type": "box"}]
        ]
    )
    
    # --- Graphique 1: Tendance des Prix (Médian) ---
    df_agg_annee = df.groupby('annee')['prix_m2'].median().reset_index()
    fig.add_trace(
        go.Scatter(
            x=df_agg_annee['annee'], 
            y=df_agg_annee['prix_m2'],
            mode='lines+markers',
            name='Prix Médian',
            line=dict(color='blue', width=3)
        ), row=1, col=1
    )
    
    # --- Graphique 2: Volume des Ventes ---
    df_vol_annee = df.groupby('annee').size().reset_index(name='transactions')
    fig.add_trace(
        go.Bar(
            x=df_vol_annee['annee'], 
            y=df_vol_annee['transactions'],
            name='Transactions',
            marker_color='lightblue'
        ), row=1, col=2
    )

    # --- Graphique 3: Types de biens (Pie Chart) ---
    df_types = df[df['type_local'].isin(['Appartement', 'Maison'])]
    type_counts = df_types['type_local'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=type_counts.index, 
            values=type_counts.values,
            hole=.3,
            name='Types',
            marker=dict(colors=['#1f77b4', '#ff7f0e'])
        ), row=2, col=1
    )
    
    # --- Graphique 4: Focus Local (Box Plot) ---
    
    # --- OPTIMISATION (SAMPLING) ---
    # Au lieu de prendre TOUTES les données, on prend un échantillon aléatoire.
    # 50 000 points sont largement suffisants pour un box plot.
    n_samples = min(50000, len(df))
    df_sample = df.sample(n_samples)
    
    # On filtre sur l'échantillon, pas sur le df complet
    df_focus = df_sample[df_sample['code_departement'].isin(['69', '75', '13', '33', '59'])] 
    
    # Création du groupe (pour l'axe X)
    def assign_group(dept):
        if dept == '69': return 'Rhône (69)'
        if dept == '75': return 'Paris (75)'
        return 'Autres (13, 33, 59)'
        
    df_focus['Groupe'] = df_focus['code_departement'].apply(assign_group)
    
    fig.add_trace(
        go.Box(
            x=df_focus['Groupe'],
            y=df_focus['prix_m2'],
            name='Distribution Prix'
        ), row=2, col=2
    )
    
    # --- Configuration du Layout ---
    fig.update_layout(
        title_text="<b>Dashboard Stratégique (Besoins Persona)</b>",
        title_x=0.5,
        height=800,
        showlegend=False,
        font=dict(family="Arial", size=12)
    )
    fig.update_yaxes(title_text="Prix Médian (€/m²)", row=1, col=1)
    fig.update_yaxes(title_text="Nb. Transactions", row=1, col=2)
    fig.update_yaxes(title_text="Prix au m²", row=2, col=2)
    
    fig.show()

# --- Exécution ---
if not df_clean.empty:
    create_persona_dashboard(df_clean)

<div style="background-color: #f0ffff; border-left: 6px solid #00BCD4; padding: 15px; border-radius: 5px;">

## 🗺️ 6. Définition de la Carte Interactive

Maintenant qu'Alexandre a une vue d'ensemble, il veut *explorer* ("Où investir ?").

Cette cellule **définit** les deux fonctions nécessaires pour la carte :

1.  `create_interactive_widgets` : Définit tous les filtres (sliders, menus déroulants).
2.  `plot_interactive_map` : La fonction de traçage qui réagit aux widgets.

*Note : L'agrégation des données se fait par **département** pour que la carte reste fluide. Une carte nationale par commune (des millions de points) n'est pas réalisable pour une exploration interactive.*

</div>

In [ ]:
def create_interactive_widgets(df):
    """
    Crée et retourne tous les widgets IPyWidgets pour le dashboard interactif.
    """
    style = {'description_width': '150px'}
    layout = Layout(width='400px')
    
    # 1. Période (Années)
    annees_disponibles = sorted(df['annee'].unique().astype(int))
    if not annees_disponibles:
        annees_disponibles = [2021, 2025] # Fallback
        
    periode_slider = widgets.SelectionRangeSlider(
        options=annees_disponibles,
        index=(0, len(annees_disponibles)-1),
        description='📅 Période (Années):',
        style=style,
        layout=Layout(width='500px')
    )
    
    # 2. Type de bien
    types_biens = ['Tous'] + sorted([t for t in df['type_local'].unique() if t in ['Maison', 'Appartement']])
    type_bien_dropdown = widgets.Dropdown(
        options=types_biens,
        value='Tous',
        description='🏠 Type de bien:',
        style=style, layout=layout
    )
    
    # 3. Métrique
    metrique_dropdown = widgets.Dropdown(
        options=[
            ('Prix médian au m²', 'median'),
            ('Prix moyen au m²', 'mean'),
            ('Nombre de transactions', 'count')
        ],
        value='median',
        description='📊 Métrique:',
        style=style, layout=layout
    )
    
    # 4. Granularité Temporelle (pour l'animation)
    agregation_dropdown = widgets.Dropdown(
        options=[('Annuelle', 'annee'), ('Trimestrielle', 'trimestre')],
        value='annee',
        description='⏰ Animation par:',
        style=style, layout=layout
    )
    
    return periode_slider, type_bien_dropdown, metrique_dropdown, agregation_dropdown

def plot_interactive_map(periode, type_bien, metrique, agregation):
    """
    Fonction de traçage appelée par 'interact'.
    Crée la carte de France interactive.
    """
    
    # 1. Filtrage des données (rapide grâce aux types optimisés)
    df_filtre = df_clean[
        (df_clean['annee'].astype(int) >= periode[0]) & 
        (df_clean['annee'].astype(int) <= periode[1])
    ]
    
    if type_bien != 'Tous':
        df_filtre = df_filtre[df_filtre['type_local'] == type_bien]
        
    if df_filtre.empty:
        print("❌ Aucune transaction ne correspond à ces filtres.")
        return

    # 2. Agrégation par département et période (pour la fluidité)
    time_col = agregation # 'annee' or 'trimestre'
    
    # Définition de la fonction d'agrégation
    agg_funcs = {
        'prix_m2': 'median' if metrique == 'median' else ('mean' if metrique == 'mean' else 'count'),
        'latitude': 'mean',
        'longitude': 'mean',
    }
    
    # Cas spécial pour 'count'
    if metrique == 'count':
        agg_funcs['prix_m2'] = 'count'

    df_agg = df_filtre.groupby(['code_departement', time_col]).agg(agg_funcs).reset_index()
    
    # Renommage pour la clarté
    df_agg = df_agg.rename(columns={'prix_m2': 'valeur_metrique', 
                                    'latitude': 'lat', 
                                    'longitude': 'lon'})
    
    # ----------------------------------------------------------------------
    # LA LIGNE QUI CONVERTIT EN STRING ÉTAIT ICI. C'ÉTAIT TROP TÔT.
    # ----------------------------------------------------------------------
    
    # 3. Définition des paramètres du graphique
    if metrique == 'count':
        metric_label = 'Nombre de transactions'
        color_scale = 'Viridis'
        size_col = 'valeur_metrique' # La taille représente le volume
    else:
        metric_label = 'Prix médian au m²' if metrique == 'median' else 'Prix moyen au m²'
        color_scale = 'RdYlBu_r' # Rouge=Cher, Bleu=Abordable
        
        # On calcule le volume total pour la taille des bulles
        # ICI : time_col est encore un INT, donc le groupby fonctionne
        size_data = df_filtre.groupby(['code_departement', time_col]).size().reset_index(name='volume')
        
        # ICI : Le merge fonctionne car les deux 'time_col' (ex: 'annee') sont des INT
        df_agg = pd.merge(df_agg, size_data, on=['code_departement', time_col])
        size_col = 'volume'
        
    # ----------------------------------------------------------------------
    # CORRECTION : ON CONVERTIT EN STRING ICI, JUSTE AVANT LE GRAPHIQUE
    # ----------------------------------------------------------------------
    # Pour l'animation, la colonne doit être de type string
    df_agg[time_col] = df_agg[time_col].astype(str)
        
    # 4. Traçage (Plotly)
    fig = px.scatter_mapbox(
        df_agg,
        lat="lat",
        lon="lon",
        color="valeur_metrique",
        size=size_col,
        hover_name="code_departement",
        hover_data={
            "code_departement": True,
            "valeur_metrique": f":.0f {'€/m²' if metrique != 'count' else ''}",
            "lat": False,
            "lon": False,
            size_col: True
        },
        animation_frame=time_col, # Maintenant, time_col est bien un string
        color_continuous_scale=color_scale,
        size_max=50,
        zoom=4.5,
        center={"lat": 46.603354, "lon": 1.888334},
        mapbox_style="carto-positron",
        title=f"<b>🗺️ Évolution du Marché Immobilier par Département</b>",
        height=700
    )
    
    fig.update_layout(
        title_x=0.5,
        font=dict(family="Arial", size=12),
        coloraxis_colorbar=dict(
            title=metric_label
        )
    )
    
    fig.show()

<div style="background-color: #f4f4f4; border-left: 6px solid #607D8B; padding: 15px; border-radius: 5px;">

## 🚀 7. Affichage du Dashboard Interactif

Cette cellule **exécute** le code de la cellule précédente.

Elle appelle `create_interactive_widgets` pour afficher les contrôles et connecte ces contrôles à la fonction `plot_interactive_map` grâce à `widgets.interactive_output`.

C'est l'exécution de cette cellule qui fait apparaître le dashboard complet (filtres + carte).

</div>

In [ ]:
# --- Exécution et Lancement de l'Interface ---
if not df_clean.empty:
    print("🎛️ Génération du dashboard interactif...")
    
    # Création des widgets
    (
        periode_slider, 
        type_bien_dropdown, 
        metrique_dropdown, 
        agregation_dropdown
    ) = create_interactive_widgets(df_clean)

    # Titre HTML
    display(HTML("""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 20px; border-radius: 15px; margin: 20px 0;'>
        <h2 style='color: white; text-align: center; margin: 0; font-size: 28px;
                   font-family: Arial Black;'>
            🗺️ Dashboard d'Exploration Géographique
        </h2>
        <p style='color: white; text-align: center; margin: 10px 0 0 0; font-size: 16px;'>
            Utilisez les filtres pour explorer les tendances sur 5 ans.
        </p>
    </div>
    """))
    
    # Organisation des widgets
    col1 = VBox([periode_slider])
    col2 = VBox([type_bien_dropdown, metrique_dropdown, agregation_dropdown])
    
    display(HBox([col1, col2]))

    # Lancement de l'interface interactive
    # C'est ici que la fonction plot_interactive_map est connectée aux widgets
    interactive_output = widgets.interactive_output(
        plot_interactive_map,
        {
            'periode': periode_slider,
            'type_bien': type_bien_dropdown,
            'metrique': metrique_dropdown,
            'agregation': agregation_dropdown
        }
    )
    
    # Affichage final de la carte
    display(interactive_output)
else:
    print("❌ Dashboard interactif non lancé (pas de données).")

<div style="background-color: #f5f5f5; border-left: 6px solid #795548; padding: 15px; border-radius: 5px;">

## 🛠️ 8. Outils d'Analyse (Export & Insights)

Enfin, nous fournissons à Alexandre les outils dont il a besoin pour son reporting (*Besoins IV.1 et IV.5*) :

1.  **Générer un Rapport d'Insights** : Un résumé textuel des KPI clés.
2.  **Exporter les Données** : Un bouton pour sauvegarder les données *agrégées* par département au format CSV/Excel.

</div>

In [ ]:
def export_analysis_data(df):
    """
    Exporte les données d'analyse (agrégées par département) avec métadonnées.
    """
    if df.empty:
        print("❌ Aucune donnée à exporter.")
        return

    print("⏳ Préparation du résumé statistique par département...")
    
    # Création d'un résumé statistique par département et année
    dept_summary = df.groupby(['annee', 'code_departement']).agg({
        'prix_m2': ['mean', 'median'],
        'valeur_fonciere': ['count', 'sum'],
        'surface_reelle_bati': 'mean'
    }).round(2)
    
    # Aplatir les MultiIndex des colonnes
    dept_summary.columns = [
        'Prix_moyen_m2', 'Prix_median_m2',
        'Nb_transactions', 'Volume_total_Euros', 'Surface_moyenne_bati'
    ]
    dept_summary = dept_summary.reset_index()
    
    # Sauvegarde en CSV et Excel
    export_filename_csv = f"analyse_dvf_departements_{datetime.now().strftime('%Y%m%d')}.csv"
    export_filename_excel = f"analyse_dvf_departements_{datetime.now().strftime('%Y%m%d')}.xlsx"
    
    try:
        dept_summary.to_csv(export_filename_csv, index=False, encoding='utf-8-sig')
        dept_summary.to_excel(export_filename_excel, index=False, sheet_name='Analyse_Departement')
        
        print(f"✅ Exportation réussie !")
        print(f"   -> Fichier CSV : {export_filename_csv}")
        print(f"   -> Fichier Excel : {export_filename_excel}")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'exportation : {e}")

def generate_insights_report(df):
    """
    Génère un rapport textuel simple avec les insights clés.
    """
    if df.empty:
        print("❌ Aucune donnée pour générer un rapport.")
        return
        
    print("🤖 Génération du rapport d'insights...")
    
    # Période
    start_year = df['annee'].min()
    end_year = df['annee'].max()
    
    # Stats globales
    global_median = df['prix_m2'].median()
    total_transactions = len(df)
    
    # Tendance
    yearly_median = df.groupby('annee')['prix_m2'].median()
    start_price = yearly_median.iloc[0]
    end_price = yearly_median.iloc[-1]
    evolution_pct = ((end_price - start_price) / start_price) * 100
    
    # --- Affichage du rapport ---
    report = f"""
    ========================================================
    RAPPORT D'ANALYSE IMMOBILIÈRE (2021-2025)
    ========================================================
    
    PÉRIODE D'ANALYSE : {start_year} à {end_year}
    TRANSACTIONS VALIDES ANALYSÉES : {total_transactions:,}
    
    --- INSIGHTS GLOBAUX ---
    Prix médian global (sur 5 ans) : {global_median:,.0f} €/m²
    
    --- TENDANCE DES PRIX (MÉDIAN) ---
    Prix en {start_year} : {start_price:,.0f} €/m²
    Prix en {end_year} : {end_price:,.0f} €/m²
    Évolution sur 5 ans : {evolution_pct:+.2f}%
    
    ================ FIN DU RAPPORT ========================
    """
    print(report)

# --- Création des Boutons ---
if not df_clean.empty:
    export_button = widgets.Button(
        description="Exporter Résumé (.csv/.xlsx)",
        button_style='success',
        icon='download',
        layout=Layout(width='300px', height='50px')
    )
    
    insights_button = widgets.Button(
        description="Générer Rapport d'Insights",
        button_style='info',
        icon='lightbulb',
        layout=Layout(width='300px', height='50px')
    )
    
    output_tools = widgets.Output() # Pour afficher le résultat des clics

    def on_export_click(b):
        with output_tools:
            clear_output()
            export_analysis_data(df_clean)
    
    def on_insights_click(b):
        with output_tools:
            clear_output()
            generate_insights_report(df_clean)
    
    export_button.on_click(on_export_click)
    insights_button.on_click(on_insights_click)
    
    display(HTML("<h2 style='color: #2c3e50; margin-top: 40px; border-bottom: 2px solid #2c3e50;'> Outils d'Analyse (Export)</h2>"))
    display(HBox([export_button, insights_button]))
    display(output_tools)